# Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Preprocessing

In [2]:
data = pd.read_csv("data/train.csv", sep = ';').to_numpy()

for i in range(len(data)):
    if data[i][1]=='white':
        data[i][1]=0
    elif data[i][1]=='red':
        data[i][1]=1
        
train = data[:3600]
valid = data[3600:]

x_train = train.T[:13].T
y_train = train[:,13]

x_valid = valid.T[:13].T
y_valid = valid[:,13]

y = []
for i in y_train:
    y.append(int(i))
y_train = y

y = []
for i in y_valid:
    y.append(int(i))
y_valid = y

x_test = pd.read_csv("data/test_public.csv", sep = ';').to_numpy()

for i in range(len(x_test)):
    if x_test[i][1]=='white':
        x_test[i][1]=0
    elif x_test[i][1]=='red':
        x_test[i][1]=1

# Model

In [26]:
params = [{'max_depth': np.arange(1,5,1), 
           'max_leaf_nodes': np.arange(1,100,10), 
           'min_samples_split': np.arange(1,10,1)}]

classifier = GridSearchCV(DecisionTreeClassifier(), params, refit=True)
classifier = classifier.fit(x_train, y_train)

predictions_valid = classifier.predict(x_valid)
accuracy_score(y_valid, predictions_valid)

C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
360 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\louis\.julia\conda\3\lib\site-packages\sklearn\tree\_classes.py", line 937, in f

0.5649419218585006

# Prediction

In [27]:
predictions = classifier.predict(x_test)
df = pd.DataFrame(predictions, columns=['quality'])
df.to_csv("submission.csv")
df

,quality
0,6
1,6
2,5
3,6
4,6
...,...
970,6
971,7
972,6
973,5
